In [6]:
n = 2
dp = [0 for _ in range(n+1)]
dp[0] = 1
dp[1] = 1
dp[2] = 2

for i in range(2, n+1):
    dp[i] = dp[i-1] + dp[i-2]                
    print(dp)

[1, 1, 2]


In [4]:
dp[i]

3

In [8]:
prices = [7,1,5,3,6,4]
dp = []
profit = 0
dp = []
for i in range(1, len(prices)):
    profit = -prices[i-1] + prices[i]
    dp.append(profit)

In [9]:
dp

[-6, 4, -2, 3, -2]

In [13]:
prices = [7,1,5,3,6,4]
dp = [[0,0] for _ in range(len(prices))]
dp[0][0] = -prices[0]
dp[0][1] = 0
for i in range(1,len(prices)):
    dp[i][0] = max(dp[i-1][0], -prices[i])
    dp[i][1] = max(dp[i-1][1], prices[i]+dp[i-1][0])    
    print(dp)

[[-7, 0], [-1, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[-7, 0], [-1, 0], [-1, 4], [0, 0], [0, 0], [0, 0]]
[[-7, 0], [-1, 0], [-1, 4], [-1, 4], [0, 0], [0, 0]]
[[-7, 0], [-1, 0], [-1, 4], [-1, 4], [-1, 5], [0, 0]]
[[-7, 0], [-1, 0], [-1, 4], [-1, 4], [-1, 5], [-1, 5]]


In [12]:
dp

[[7, 0], [7, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

In [7]:
import sys
import math
from io import StringIO

# 构造输入数据（你提供的 Sample Input）
sample_input = """5 1
#....
.....
..##.
..##.
.....
"""

# 将输入重定向到 sys.stdin
sys.stdin = StringIO(sample_input)

# 主函数（不需要改动）
def main():
    input = sys.stdin.readline
    N, P = map(int, input().split())
    grid = [input().strip() for _ in range(N)]

    # Build prefix sums of '#' → 1, '.' → 0
    M = [[0] * (N+1) for _ in range(N+1)]
    for i in range(N):
        for j in range(N):
            M[i+1][j+1] = (1 if grid[i][j]=='#' else 0)
    for i in range(1, N+1):
        for j in range(1, N+1):
            M[i][j] += M[i-1][j] + M[i][j-1] - M[i-1][j-1]

    def rect_sum(x1, y1, x2, y2):
        return (M[x2][y2]
                - M[x1-1][y2]
                - M[x2][y1-1]
                + M[x1-1][y1-1])

    total_hash = M[N][N]
    N2 = N*N
    p = P / 100.0
    log_p   = math.log(p)
    log_1p  = math.log(1-p)
    logN    = math.log(N)

    best_score = -1e300
    best_l = best_x = best_y = 0

    # Precompute -2*log(N-l+1) for each l
    prior_cache = [ -logN - 2*math.log(N - l + 1) if l != 0 else 0 for l in range(N+1) ]

    for l in range(1, N+1):
        l2 = l*l
        prior = prior_cache[l]
        for x in range(1, N-l+2):
            x2 = x + l - 1
            for y in range(1, N-l+2):
                y2 = y + l - 1
                a = rect_sum(x, y, x2, y2)
                c = total_hash - a
                ll = ((N2 - l2 + a - c) * log_1p
                      + (l2 - a + c) * log_p)
                score = ll + prior
                if score > best_score:
                    best_score, best_l, best_x, best_y = score, l, x, y

    total = 0.0
    for l in range(1, N+1):
        l2 = l*l
        prior = prior_cache[l]
        for x in range(1, N-l+2):
            x2 = x + l - 1
            for y in range(1, N-l+2):
                y2 = y + l - 1
                a = rect_sum(x, y, x2, y2)
                c = total_hash - a
                ll = ((N2 - l2 + a - c) * log_1p
                      + (l2 - a + c) * log_p)
                total += math.exp(ll + prior - best_score)

    Pmax = 1.0 / total

    bx, by, bl = best_x-1, best_y-1, best_l
    for i in range(N):
        row = []
        for j in range(N):
            if bx <= i < bx+bl and by <= j < by+bl:
                row.append('#')
            else:
                row.append('.')
        print(''.join(row))
    print(f"{Pmax:.8f}")

# 调用主函数
main()

.....
.....
..##.
..##.
.....
0.99999666


In [9]:
# Example of Block Floating Exponent Unification in Jupyter

# 1) Define the example inputs exactly as given:
example_lines = [
    "0 10000011 11010000000000000000000",
    "1 10001000 11010000000000000000000",
    "1 10000000 11010000000000000000000",
    "0 00000001 11010000000000000000000"
]

# 2) Block-floating processing function
def process_block_floating(lines):
    """
    lines: list of strings, each "sign exp mantissa"
    returns: list of transformed strings "sign exp mantissa"
    """
    # Parse inputs
    inputs = []
    for line in lines:
        s_str, e_str, m_str = line.split()
        inputs.append((s_str, int(e_str, 2), int(m_str, 2)))

    # Unify exponent to the maximum
    _, exponents, _ = zip(*inputs)
    E_max = max(exponents)
    E_max_bin = format(E_max, '08b')

    # Process each
    outputs = []
    for sign_str, e_i, m_i in inputs:
        d = E_max - e_i
        numerator = (1 << 23) + m_i     # include hidden 1
        shift = d + 1                   # total right shift
        q, r = divmod(numerator, 1 << shift)

        # Round: if remainder*2 > divisor, round up; ties go to zero (keep q)
        if r * 2 > (1 << shift):
            m_out = q + 1
        else:
            m_out = q

        out_line = f"{sign_str} {E_max_bin} {format(m_out, '023b')}"
        outputs.append(out_line)

    return outputs

# 3) Run and display
output_lines = process_block_floating(example_lines)
print("Input:")
print("\n".join(example_lines))
print("\nOutput:")
print("\n".join(output_lines))

Input:
0 10000011 11010000000000000000000
1 10001000 11010000000000000000000
1 10000000 11010000000000000000000
0 00000001 11010000000000000000000

Output:
0 10001000 00000111010000000000000
1 10001000 11101000000000000000000
1 10001000 00000000111010000000000
0 10001000 00000000000000000000000


In [ ]:
import sys
import math


def parse_input(lines):
    """
    Parse the first line to get N (grid size) and P, then the next N lines to construct the grid.
    
    Parameters:
     N - Size of grid cell
     P - Noise probability in percentage (e.g., 60 means 60%).
     grid - The final state of 2D character grid with '.'  and '#'.
     
    Returns:
     N, P, grid array in the test cases
    """
    
    N, P = map(int, lines[0].split())
    grid = [list(lines[i + 1]) for i in range(N)]
    return N, P, grid


def build_prefix_sum(N, grid):
    """
    Build a 2D prefix sum matrix for '#' cells to enable O(1) region sum queries.
    
    Parameters:
     N - Size of grid cell
     grid - The final state of grid
     
    Returns:
     2D prefix_sum array that contains values based on the '#' and '.'
    """
    
    # Create (N+1)x(N+1) zero matrix to simplify boundary conditions (1-based index)
    prefix_sum = [[0] * (N + 1) for _ in range(N + 1)]

    # Convert '#' to 1 and '.' to 0 and populate prefix_sum
    for i in range(N):
        for j in range(N):
            prefix_sum[i + 1][j + 1] = 1 if grid[i][j] == '#' else 0

    # Accumulate values to build 2D prefix sum
    for i in range(1, N + 1):
        for j in range(1, N + 1):
            prefix_sum[i][j] += (
                prefix_sum[i - 1][j] +
                prefix_sum[i][j - 1] -
                prefix_sum[i - 1][j - 1]
            )
    return prefix_sum


def rect_sum(M, x1, y1, x2, y2):
    """
    Query the number of '#' inside rectangle from (x1,y1) to (x2,y2) using the 2D prefix sum matrix
    
    Parameters:
     M - The 2D prefix sum matrix from build_prefix_sum fuction
     x1 - Top row corrdinate 
     y1 - Left column corrdinate
     x2 - Bottom row corrdinate
     y2 - Right column corrdinate
     
    Returns:
     The number of '#' inside the specific rectangle
    """
    
    return M[x2][y2] - M[x1 - 1][y2] - M[x2][y1 - 1] + M[x1 - 1][y1 - 1]


def compute_log_posterior(N, P, grid, M):
    """
    Compute the log posterior probability for all possible square regions in the grid
    and return the one with the highest posterior score using a Bayesian MAP approach.
    
    Parameters:
     N - Size of the grid (grid is N x N).
     P - Noise probability
     grid - 2D character grid with '.'  and '#'.
     M - 2D prefix sum matrix for '#' cells.
    
    Returns:
     best_x - Row index (1-based) of the top-left corner of the best square region.
     best_y - Column index (1-based) of the top-left corner of the best square region.
     best_l - Side length of the best square region.
     best_score - Maximum posterior score (log-likelihood + log-prior).
     prior_cache - Cached prior log probabilities for each square size (indexed by side length).
    
    """
    
    total_hash = M[N][N]          # Total number of '#' in the whole grid
    N2 = N * N                    # Total number of cells
    p = P / 100.0                 # Convert P to probability
    log_p = math.log(p)           # log(p)
    log_1p = math.log(1 - p)      # log(1-p)
    logN = math.log(N)

    best_score = -math.inf        # Initialize best score
    best_x = best_y = best_l = 0  # To store best region's coordinates and size

    # Cache prior scores to penalize large regions
    prior_cache = [
        -logN - 2 * math.log(N - l + 1) if l != 0 else 0
        for l in range(N + 1)
    ]

    # Try every square region of size l x l
    for l in range(1, N + 1):
        l2 = l * l
        prior = prior_cache[l]

        for x in range(1, N - l + 2):
            x2 = x + l - 1
            for y in range(1, N - l + 2):
                y2 = y + l - 1

                a = rect_sum(M, x, y, x2, y2)     # number of '#' inside region
                c = total_hash - a               # number of '#' outside region

                # log-likelihood (how well this region explains the '#'s)
                ll = ((N2 - l2 + a - c) * log_1p + (l2 - a + c) * log_p)
                score = ll + prior               # posterior = likelihood + prior

                # Keep the best scoring region
                if score > best_score:
                    best_score = score
                    best_l = l
                    best_x = x
                    best_y = y

    return best_x, best_y, best_l, best_score, prior_cache

def compute_normalization(N, P, M, total_hash, best_score, prior_cache):
    
    """
    Compute the normalized probability of the best region via log-sum-exp trick
    
    This function applies softmax normalization across all possible square regions
    to compute the posterior probability (in relative terms) of the region with the best score.
    
    Parameters:
     N - Size of the grid .
     P - Pollution probability.
     M - 2D prefix sum matrix.
     total_hash - Total number of '#' symbols in the entire grid.
     best_score - The highest posterior score found among all square regions.
     prior_cache - Cached log prior probabilities for each square size.

    Returns:
     Normalized posterior probability of the best-scoring region.

    """
    
    total = 0.0
    N2 = N * N
    p = P / 100.0
    log_p = math.log(p)
    log_1p = math.log(1 - p)

    # Iterate over all possible square regions and compute their exp(score - best_score)
    for l in range(1, N + 1):
        l2 = l * l
        prior = prior_cache[l]

        for x in range(1, N - l + 2):
            x2 = x + l - 1
            for y in range(1, N - l + 2):
                y2 = y + l - 1

                a = rect_sum(M, x, y, x2, y2)
                c = total_hash - a
                ll = ((N2 - l2 + a - c) * log_1p + (l2 - a + c) * log_p)

                # log-sum-exp component for softmax normalization
                total += math.exp(ll + prior - best_score)

    return 1.0 / total  # final normalized probability

# Generate the output grid with '#' marking the most probable polluted region
def generate_output_grid(N, best_x, best_y, best_l):
    """
    Generate a grid marking the most probable polluted region with '#'.

    This function creates a new N x N character grid where the best region is filled
    with '#' symbols and the rest of the grid is filled with '.' symbols.

    Parameters:
     N - Size of the output grid.
     best_x - Row index of the top-left corner of the best region.
     best_y - Column index of the top-left corner of the best region.
     best_l - Side length of the best square region.

    Returns:
     List of N strings, each representing one row of the output grid.
    """
    bx, by, bl = best_x - 1, best_y - 1, best_l  # adjust to 0-based index
    output = []

    for i in range(N):
        row = []
        for j in range(N):
            if bx <= i < bx + bl and by <= j < by + bl:
                row.append('#')  # inside best region
            else:
                row.append('.')  # outside region
        output.append(''.join(row))

    return output

# Main entry: parse input, run computation, and print results
def main(lines):
    N, P, grid = parse_input(lines)
    M = build_prefix_sum(N, grid)
    best_x, best_y, best_l, best_score, prior_cache = compute_log_posterior(N, P, grid, M)
    Pmax = compute_normalization(N, P, M, M[N][N], best_score, prior_cache)
    result_grid = generate_output_grid(N, best_x, best_y, best_l)

    # Output grid
    for row in result_grid:
        print(row)

    # Output probability (formatted to 8 decimal places)
    print(f"{Pmax:.8f}")


if __name__ == '__main__':
    lines = [line.rstrip('\n') for line in sys.stdin]
    main(lines)


\begin{cases}
1, & \text{if } \texttt{grid}[i][j] = \# \\
0, & \text{if } \texttt{grid}[i][j] = \texttt{.}
\end{cases}


In [14]:
2**4 * 1.625 / 2**9

0.05078125

In [15]:
format(0.05078125, '08b')

ValueError: Unknown format code 'b' for object of type 'float'

In [ ]:
0 10000011 11010000000000000000000
1 10001000 11010000000000000000000
1 10000000 11010000000000000000000
0 00000001 11010000000000000000000

In [ ]:
0 10001000 00000111010000000000000
1 10001000 11101000000000000000000
1 10001000 00000000111010000000000
0 10001000 00000000000000000000000

In [ ]:
for sign, e_i, m_i in zip(signs, exponents, mantissas):
    d = E_max - e_i                     # 1. 当前指数和最大指数的差值
    numerator = (1 << 23) + m_i        # 2. 添加隐含的 '1'，构造完整尾数（24位）
    shift = d + 1                      # 3. 因为指数差 d，尾数需要右移 d+1 位
    m_out = numerator >> shift         # 4. 对尾数做右移，得到对齐后的新尾数

In [16]:
exponents = [10000011,10001000]
mantissas = [11010000000000000000000, 11010000000000000000000]

In [17]:
max_E = max(exponents)

In [19]:
max_E - exponents[0]

989

In [20]:
(1 << 23) + 11010000000000000000000

11010000000000008388608

In [21]:
#(1 << 23) = 1000000000000000000

8388608

In [22]:
(1 << 23)

8388608

In [23]:
max_E - exponents[0] + 1

990

In [25]:
numerator = (1 << 23) + 11010000000000000000000

In [26]:
shift = max_E - exponents[0] + 1

In [27]:
m_out = numerator >> shift

In [48]:
format(2752500000000002097152,'023b')

'100101010011011010011100011001101010000101000110110100100000000000000000'

In [ ]:
01110100000000000000000
00000111010000000000000

In [51]:
int(11010000000000000000000)

11010000000000000000000

In [52]:
1 << 23

8388608

In [53]:
print(f"{3801088:023b}")

01110100000000000000000
